In [5]:
gsutil -m rm gs://lpr-cdip-words2/* > log 2>&1 || true
tail log

CommandException: 1 files/objects could not be removed.


In [6]:
kubectl delete jobs --all 2>&1 | tail 
kubectl delete pods --all 2>&1 | tail
kubectl delete services shuffle || true

job.batch "mapper90" deleted
job.batch "mapper91" deleted
job.batch "mapper92" deleted
job.batch "mapper93" deleted
job.batch "mapper94" deleted
job.batch "mapper95" deleted
job.batch "mapper96" deleted
job.batch "mapper97" deleted
job.batch "mapper98" deleted
job.batch "mapper99" deleted
No resources found
service "shuffle" deleted


In [7]:
cat > kubetpl.yaml <<'EOF'
image: gcr.io/research-191823/simple
memory: 8G
app: shuffle
subdomain: shuffle
EOF

# Name resolution between pods requires a headless service with an app selector to be run; selected pods will get name-resolved.
kubetpl service | kubectl apply -f -

service/shuffle created


In [8]:
src=gs://lpr-cdip-words/*.tgz
dst=gs://lpr-cdip-words2
out=$dst/cdip-words
image=gcr.io/research-191823/simple

echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out/* | wc -l

src=648
dst=0
out=CommandException: One or more URLs matched no objects.
0


In [9]:
for index in {00..19}; do
kubetpl pod -C 2 -M 16G -n writer$index -c "
hostname; date
tarsplit zpull://0.0.0.0:10000 -s 1e9 -n 1e6 -z -o '|gsutil cp - $dst/TEMP-$index-{shard:06d}.tgz'
hostname; date
" | kubectl apply -f -
done

pod/writer00 created
pod/writer01 created
pod/writer02 created
pod/writer03 created
pod/writer04 created
pod/writer05 created
pod/writer06 created
pod/writer07 created
pod/writer08 created
pod/writer09 created
pod/writer10 created
pod/writer11 created
pod/writer12 created
pod/writer13 created
pod/writer14 created
pod/writer15 created
pod/writer16 created
pod/writer17 created
pod/writer18 created
pod/writer19 created


In [10]:
gsutil ls $src | sort -R | cat -n |
while read index shard; do
kubetpl job -M 8G -L jobgroup=readers -I $image -n reader$index -c "
gsutil cat $shard | tarscat --shuffle 10000 -o 'zpush://writer{00..19}.shuffle:10000'
" | kubectl apply -f -
done

job.batch/reader1 created
job.batch/reader2 created
job.batch/reader3 created
job.batch/reader4 created
job.batch/reader5 created
job.batch/reader6 created
job.batch/reader7 created
job.batch/reader8 created
job.batch/reader9 created
job.batch/reader10 created
job.batch/reader11 created
job.batch/reader12 created
job.batch/reader13 created
job.batch/reader14 created
job.batch/reader15 created
job.batch/reader16 created
job.batch/reader17 created
job.batch/reader18 created
job.batch/reader19 created
job.batch/reader20 created
job.batch/reader21 created
job.batch/reader22 created
job.batch/reader23 created
job.batch/reader24 created
job.batch/reader25 created
job.batch/reader26 created
job.batch/reader27 created
job.batch/reader28 created
job.batch/reader29 created
job.batch/reader30 created
job.batch/reader31 created
job.batch/reader32 created
job.batch/reader33 created
job.batch/reader34 created
job.batch/reader35 created
job.batch/reader36 created
job.batch/reader37 created
job.batch/

In [11]:
sleep 60
kubectl logs job.batch/reader1 | tail

# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 580000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 590000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 600000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 610000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 620000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 630000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 640000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 650000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 660000
# send <tarproclib.zcom.MultiWriter object at 0x7f9fab285828> 670000


In [15]:
kubectl logs writer01 | tail

# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1780000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1790000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1800000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1810000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1820000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1830000            
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1840000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1850000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1860000
# recv <tarproclib.zcom.Connection object at 0x7f10149376a0> 1870000


In [16]:
date

Sun Nov 17 18:10:21 PST 2019


In [21]:
kubectl wait jobs -l jobgroup=readers --for=condition=complete --timeout=-1s

job.batch/reader1 condition met
job.batch/reader10 condition met
job.batch/reader100 condition met
job.batch/reader101 condition met
job.batch/reader102 condition met
job.batch/reader103 condition met
job.batch/reader104 condition met
job.batch/reader105 condition met
job.batch/reader106 condition met
job.batch/reader107 condition met
job.batch/reader108 condition met
job.batch/reader109 condition met
job.batch/reader11 condition met
job.batch/reader110 condition met
job.batch/reader111 condition met
job.batch/reader112 condition met
job.batch/reader113 condition met



In [22]:
date

Mon Nov 18 07:33:21 PST 2019


In [23]:
echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out/* | wc -l

src=648
dst=1320
out=CommandException: One or more URLs matched no objects.
0


In [26]:
sleep 3
kubectl get pods | grep Running

: 1

In [24]:
kubectl exec writer00 -- tarscat --nodata --eof -o 'zpush://writer{00..19}.shuffle:10000'

# got 0 files
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f54a01f1dc8> zpush://writer00.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f54a01f1e88> zpush://writer01.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f54a01f1f48> zpush://writer02.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f975048> zpush://writer03.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f975108> zpush://writer04.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f9751c8> zpush://writer05.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f975288> zpush://writer06.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f975348> zpush://writer07.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f975408> zpush://writer08.shuffle:10000
# zmq_connect <zmq.sugar.socket.Socket object at 0x7f549f9754c8> zpush://writer09.shuffle:10000
# zmq_connect <zmq.sugar.s

In [25]:
sleep 3
kubectl get pods | grep Running

: 1

In [29]:
gsutil ls $dst/TEMP-*.tgz | awk '{printf("gsutil mv %s '$out'-%06d.tgz\n", $1, count+0); count+=1}' | bash -x

+ gsutil mv gs://lpr-cdip-words2/TEMP-00-000000.tgz gs://lpr-cdip-words2/cdip-words-000000.tgz
Copying gs://lpr-cdip-words2/TEMP-00-000000.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-cdip-words2/TEMP-00-000000.tgz...                             
-
Operation completed over 1 objects/244.3 MiB.                                    
+ gsutil mv gs://lpr-cdip-words2/TEMP-00-000001.tgz gs://lpr-cdip-words2/cdip-words-000001.tgz
Copying gs://lpr-cdip-words2/TEMP-00-000001.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-cdip-words2/TEMP-00-000001.tgz...                             

Operation completed over 1 objects/368.8 MiB.                                    
+ gsutil mv gs://lpr-cdip-words2/TEMP-00-000002.tgz gs://lpr-cdip-words2/cdip-words-000002.tgz
Copying gs://lpr-cdip-words2/TEMP-00-000002.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-cdip-words2/TEMP-00-000002.tgz...                             
-
Operation completed over 1 obj

In [31]:
echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out* | wc -l

src=648
dst=1339
out=1339
